In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle

In [ ]:
class PatternInterpolationTest(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        self.param('length of training pattern', len_train=500),
        self.param('number of epochs', n_epoch=3),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        seed=param.p_seed
        dt = 0.001
        target_dt = 1.0/30
        D = param.n_dims
        theta = 15
        
        dec=np.load('Training_Out\%s_decoder.npy' % seed)
        pool_size_out=int(np.load('Training_Out\%s_node_pool.npy' % seed))
        
        high_train_pca=np.load('Training_Out\%s_high_train_pca.npy' % seed)
        low_train_pca=np.load('Training_Out\%s_low_train_pca.npy' % seed)
        
        high_test_pca2=np.load('Training_Out\%s_high_test_pca2.npy' % seed)
        low_test_pca2=np.load('Training_Out\%s_low_test_pca2.npy' % seed)
        mid_test_pca2=np.load('Training_Out\%s_mid_test_pca2.npy' % seed)
        
        high_test_time=np.load('Training_Out\%s_high_test_time.npy' % seed)
        low_test_time=np.load('Training_Out\%s_low_test_time.npy' % seed)
        mid_time=np.load('Training_Out\%s_mid_time.npy' % seed)
        
############ TESTING WITH 20% ############ 
        print('Testing with 20% Testing Data')
        a_high=[]

        for j in range(20):
            print('Testing High Engagement Pattern ', int(1+j))
            testing = np.vstack([high_test_pca2[j]])

            test_net = nengo.Network(seed=seed)#param.seed)
            with test_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([high_train_pca[:,i], low_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=pool_size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                high_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(test_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[high_test_result]

            a_high.append(v)
            
        output_high_test = {}
        for i in range(len(a_high)):
            output_high_test[i]={"time":high_test_time[i], "output":a_high[i]}

        for j in range(len(output_high_test)):        
            key_j = 'output_high_test{}'.format("%03d" % j)  
            output_high_test[key_j] = output_high_test.pop(j) 
            
############################   

        a_low=[]
        for j in range(17):
            print('Testing Low Engagement Pattern ', int(1+j))
            testing = np.vstack([low_test_pca2[j]])

            test_net = nengo.Network(seed=seed)#param.seed)
            with test_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([high_train_pca[:,i], low_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=pool_size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                low_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(test_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[low_test_result]

            a_low.append(v)

        output_low_test = {}
        for i in range(len(a_low)):
            output_low_test[i]={"time":low_test_time[i], "output":a_low[i]}

        for j in range(len(output_low_test)):        
            key_j = 'output_low_test{}'.format("%03d" % j)  
            output_low_test[key_j] = output_low_test.pop(j) 

############ TESTING WITH INTERMEDIATE PATTERNS ############ 
        a_mid=[]

        for j in range(50):
            print('Testing Intermediate Pattern ', int(1+j))
            testing = np.vstack([mid_test_pca2[j]])

            mid_net = nengo.Network(seed=seed)#param.seed)
            with mid_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([high_train_pca[:,i], low_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=pool_size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                mid_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(mid_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[mid_test_result]

            a_mid.append(v)

        output_mid = {}
        for i in range(len(a_mid)):
            output_mid[i]={"time":mid_time[i], "output":a_mid[i]}

        for j in range(len(output_mid)):        
            key_j = 'output_mid{}'.format("%03d" % j)  
            output_mid[key_j] = output_mid.pop(j) 

############ SAVE DATA ############ 
       
        results = {**output_high_test, **output_low_test, **output_mid}

        return results  

In [ ]:
for seed in range(20):
    PatternInterpolationTest().run(seed=seed, p_seed=seed, data_dir='Testing_Out', data_format="npz")